In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

# houses my function to connect to Codeup DB
import wrangle as wr

**Exercises:**

- Only include properties with a transaction in 2017
    - include only the last transaction for each property 
    - zestimate error
    - date of transaction
- Only include properties that have a latitude and longitude value

**Domain Notes:**

1. Are there requirements for bedrooms for single family units?
* Yes, there is a minimum amount of bedrooms required for single family residential in Los Angeles County. According to the Los Angeles County Building Code, a single family dwelling must have at least one habitable room that is at least 120 square feet in area and has a minimum dimension of 7 feet in any direction. Additionally, the dwelling must have at least one bedroom for every two occupants, with a minimum of one bedroom per dwelling unit. 
    * Specifically, the requirements for habitable rooms and bedrooms can be found in Section 1208.4 and Section 1208.5, respectively. https://dpw.lacounty.gov/bsd/building-code/
    
2. Are there requirements for bathrooms for single family units?
* According to the Los Angeles County Building Code, a single family dwelling must have at least one bathroom that includes a toilet, sink, and bathtub or shower.
    * Specifically, the requirements for bathrooms can be found in Section 1208.2. Los angeles county adopted the california plumbing code requirements.
        * LA County: https://dpw.lacounty.gov/bsd/building-code/
        * CA Plumbing Code: 
    
3. Are there lot size requirements for a single family home?
* The lot size range is 1,500 - 5,000 square feet
    * https://planning.lacounty.gov/zoning-ordinance/
    
  

In [5]:
# set query to SQL
query = """SELECT *
FROM properties_2017
JOIN predictions_2017 using (parcelid)
LEFT JOIN airconditioningtype using (airconditioningtypeid)
LEFT JOIN architecturalstyletype using (architecturalstyletypeid)
LEFT JOIN buildingclasstype using (buildingclasstypeid)
LEFT JOIN heatingorsystemtype using (heatingorsystemtypeid)
LEFT JOIN storytype using (storytypeid)
LEFT JOIN typeconstructiontype using (typeconstructiontypeid)
WHERE propertylandusetypeid like 260 or 261 or 262 or 263 or 264 or 265 or 266 or 268 or 269 or 270 or 275 or 276 or 279
and lotsizesquarefeet >= 1500
and bathroomcnt > 1
and bedroomcnt > 1
ORDER BY parcelid;"""

In [104]:
# acquire
df = wr.get_df('zillow', query)

CSV file found and loaded


In [105]:
# look at the data
df.head()

,typeconstructiontypeid,storytypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,bathroomcnt,...,censustractandblock,id.1,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,storydesc,typeconstructiondesc
0,NaN,NaN,2.0,NaN,NaN,NaN,10711855,1087254,NaN,2.0,...,6.037113e+13,55006,-0.007357,2017-07-07,NaN,NaN,NaN,Central,NaN,NaN
1,NaN,NaN,2.0,NaN,NaN,1.0,10711877,1072280,NaN,2.0,...,6.037113e+13,71382,0.021066,2017-08-29,Central,NaN,NaN,Central,NaN,NaN
2,NaN,NaN,2.0,NaN,NaN,1.0,10711888,1340933,NaN,2.0,...,6.037113e+13,23209,0.077174,2017-04-04,Central,NaN,NaN,Central,NaN,NaN
3,NaN,NaN,2.0,NaN,NaN,NaN,10711910,1878109,NaN,2.0,...,6.037113e+13,18017,-0.041238,2017-03-17,NaN,NaN,NaN,Central,NaN,NaN
4,NaN,NaN,2.0,NaN,NaN,NaN,10711923,2190858,NaN,2.0,...,6.037113e+13,20378,-0.009496,2017-03-24,NaN,NaN,NaN,Central,NaN,NaN


In [106]:
# checking types and nulls
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52442 entries, 0 to 52441
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   typeconstructiontypeid        76 non-null     float64
 1   storytypeid                   47 non-null     float64
 2   heatingorsystemtypeid         33936 non-null  float64
 3   buildingclasstypeid           0 non-null      float64
 4   architecturalstyletypeid      70 non-null     float64
 5   airconditioningtypeid         13638 non-null  float64
 6   parcelid                      52442 non-null  int64  
 7   id                            52442 non-null  int64  
 8   basementsqft                  47 non-null     float64
 9   bathroomcnt                   52442 non-null  float64
 10  bedroomcnt                    52442 non-null  float64
 11  buildingqualitytypeid         33741 non-null  float64
 12  calculatedbathnbr             52305 non-null  float64
 13  d

In [107]:
# shape of df
df.shape

(52442, 68)

In [108]:
# numerical descriptive stats transposed to see all columns
df.describe().T

,count,mean,std,min,25%,50%,75%,max
typeconstructiontypeid,76.0,5.973684e+00,2.294157e-01,4.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00
storytypeid,47.0,7.000000e+00,0.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00
heatingorsystemtypeid,33936.0,3.966378e+00,2.562521e+00,1.000000e+00,2.000000e+00,2.000000e+00,7.000000e+00,2.400000e+01
buildingclasstypeid,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
architecturalstyletypeid,70.0,7.100000e+00,2.665670e+00,2.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,2.100000e+01
airconditioningtypeid,13638.0,2.438041e+00,3.846176e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.300000e+01
parcelid,52442.0,1.299912e+07,3.411412e+06,1.071186e+07,1.151030e+07,1.257766e+07,1.412951e+07,1.676878e+08
id,52442.0,1.497288e+06,8.594413e+05,3.490000e+02,7.579560e+05,1.500836e+06,2.241564e+06,2.982270e+06
basementsqft,47.0,6.789787e+02,7.118252e+02,3.800000e+01,2.635000e+02,5.120000e+02,8.095000e+02,3.560000e+03
bathroomcnt,52442.0,2.299397e+00,1.022764e+00,0.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00,1.800000e+01


**Acquire Notes:**

Wow. What a mess. 

* Remove buildingclasstypeid, finishedsquarefeet13, finishedsquarefeet15, buildingclassdesc off the bat because these columns contain no information at all.
* Make null function to look by row, return a df

**Exercise:**
- Only include properties with a transaction in 2017
    - include only the last transaction for each property
    - zestimate error
    - date of transaction
- Only include properties that have a latitude and longitude value

**Prepare:**
- Remove columns with no data
- Remove transactions from 2018
- Check duplicates
- Zestimate Error
- Date not null
- Lat and Long is populated


In [109]:
# remove columns with no data
df.drop(columns=['buildingclasstypeid','finishedsquarefeet13','finishedsquarefeet15','buildingclassdesc'], inplace=True)
df.head()

,typeconstructiontypeid,storytypeid,heatingorsystemtypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,bathroomcnt,bedroomcnt,...,taxdelinquencyyear,censustractandblock,id.1,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,heatingorsystemdesc,storydesc,typeconstructiondesc
0,NaN,NaN,2.0,NaN,NaN,10711855,1087254,NaN,2.0,3.0,...,NaN,6.037113e+13,55006,-0.007357,2017-07-07,NaN,NaN,Central,NaN,NaN
1,NaN,NaN,2.0,NaN,1.0,10711877,1072280,NaN,2.0,4.0,...,NaN,6.037113e+13,71382,0.021066,2017-08-29,Central,NaN,Central,NaN,NaN
2,NaN,NaN,2.0,NaN,1.0,10711888,1340933,NaN,2.0,4.0,...,NaN,6.037113e+13,23209,0.077174,2017-04-04,Central,NaN,Central,NaN,NaN
3,NaN,NaN,2.0,NaN,NaN,10711910,1878109,NaN,2.0,3.0,...,NaN,6.037113e+13,18017,-0.041238,2017-03-17,NaN,NaN,Central,NaN,NaN
4,NaN,NaN,2.0,NaN,NaN,10711923,2190858,NaN,2.0,4.0,...,NaN,6.037113e+13,20378,-0.009496,2017-03-24,NaN,NaN,Central,NaN,NaN


In [110]:
# remove transactions from 2018 - there are none
df[df.transactiondate == '%2018%']

,typeconstructiontypeid,storytypeid,heatingorsystemtypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,bathroomcnt,bedroomcnt,...,taxdelinquencyyear,censustractandblock,id.1,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,heatingorsystemdesc,storydesc,typeconstructiondesc


In [111]:
# check duplicates for parcelid - there are 122!
df[df.parcelid.duplicated()]

,typeconstructiontypeid,storytypeid,heatingorsystemtypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,bathroomcnt,bedroomcnt,...,taxdelinquencyyear,censustractandblock,id.1,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,heatingorsystemdesc,storydesc,typeconstructiondesc
258,NaN,NaN,2.0,NaN,NaN,10722858,16179,NaN,2.0,4.0,...,NaN,6.037135e+13,14034,-0.172843,2017-07-28,NaN,NaN,Central,NaN,NaN
489,NaN,NaN,2.0,NaN,NaN,10732347,1836115,NaN,2.0,4.0,...,NaN,6.037137e+13,13914,-0.221145,2017-07-25,NaN,NaN,Central,NaN,NaN
625,NaN,NaN,2.0,NaN,1.0,10739478,2119208,NaN,3.0,4.0,...,NaN,6.037800e+13,2905,-0.262967,2017-03-31,Central,NaN,Central,NaN,NaN
1133,NaN,NaN,2.0,NaN,1.0,10779619,210164,NaN,3.0,3.0,...,15.0,6.037133e+13,19107,-0.286966,2017-08-28,Central,NaN,Central,NaN,NaN
1633,NaN,NaN,2.0,NaN,1.0,10811539,449181,NaN,4.0,4.0,...,NaN,6.037138e+13,21345,-0.021841,2017-08-30,Central,NaN,Central,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50199,NaN,NaN,NaN,NaN,NaN,17193966,1835607,NaN,4.5,5.0,...,NaN,6.111006e+13,15219,-0.026852,2017-04-14,NaN,NaN,NaN,NaN,NaN
50872,NaN,NaN,NaN,NaN,NaN,17225336,2684934,NaN,3.0,4.0,...,NaN,6.111008e+13,33865,-0.100843,2017-08-30,NaN,NaN,NaN,NaN,NaN
51980,NaN,NaN,NaN,NaN,NaN,17280166,2595690,NaN,3.0,4.0,...,NaN,6.111007e+13,38851,-0.116808,2017-06-15,NaN,NaN,NaN,NaN,NaN
52036,NaN,NaN,NaN,NaN,NaN,17282392,2938730,NaN,2.0,3.0,...,NaN,6.111007e+13,64278,0.901074,2017-08-25,NaN,NaN,NaN,NaN,NaN


In [125]:
# keeping the last duplicates for parcelid
df[df.parcelid.duplicated()].sort_values(by='parcelid')
df.drop_duplicates(subset=['parcelid'], keep='last', inplace=True)

In [126]:
# verifying drop of dupes
df.parcelid.duplicated().sum()

0

In [127]:
# look at shape
df.shape

(52320, 64)

In [133]:
# check zestimate error has values
df.logerror.isnull().sum()

0

In [131]:
# check that transaction date has values
df.transactiondate.isnull().sum()

0

In [132]:
# check that lat/long has values
df.latitude.isnull().sum(), df.longitude.isnull().sum()

(0, 0)

In [134]:
# use the function to look at nulls per row 

# set variables first

num_missing = df.isnull().sum(axis=1)
pct_miss = ((num_missing / df.shape[1]) * 100).round(0)

In [137]:
# making it readable by using a dataframe
rows_missing = pd.DataFrame({'parcelid':df.parcelid,'num_cols_missing': num_missing, 'percent_cols_missing': pct_miss})

In [138]:
# look at the info
rows_missing.sort_values(by="num_cols_missing", ascending=False)

,parcelid,num_cols_missing,percent_cols_missing
52441,167687839,44,69.0
52440,167686999,44,69.0
50123,17188862,43,67.0
46567,14460039,42,66.0
43810,14341510,41,64.0
...,...,...,...
39317,14128839,20,31.0
34171,13851844,20,31.0
50644,17215105,20,31.0
46781,14465914,19,30.0


In [148]:
# preserve the unique parcelids
null_rows = df.merge(rows_missing,
                        left_index=True,
                        right_index=True).reset_index()[['parcelid_x', 'num_cols_missing', 'percent_cols_missing']]

null_rows.rename(columns={'parcelid_x':'id'}, inplace=True)

In [149]:
# sort by biggest amount of missing values
null_rows.sort_values(by="num_cols_missing", ascending=False)

,id,num_cols_missing,percent_cols_missing
52319,167687839,44,69.0
52318,167686999,44,69.0
50006,17188862,43,67.0
46456,14460039,42,66.0
43703,14341510,41,64.0
...,...,...,...
39214,14128839,20,31.0
34084,13851844,20,31.0
50526,17215105,20,31.0
46670,14465914,19,30.0


In [47]:
# make function to look at number missing per row
def nulls_by_row(df, index_id = 'parcelid'):
    """This function takes in a df and returns a df that contains all null information:
    number of missing values and the percent.
    ---
    Format: df_nulls = function()
    """
    # assign variables
    num_missing = df.isnull().sum(axis=1)
    pct_miss = ((num_missing / df.shape[1]) * 100).round(0)   
    
    # make df
    rows_missing = pd.DataFrame({'parcelid':df.parcelid,'num_cols_missing': num_missing, 'percent_cols_missing': pct_miss})
   
    # preserve the unique parcelids
    null_rows = df.merge(rows_missing,
                            left_index=True,
                            right_index=True).reset_index()[['parcelid_x', 'num_cols_missing', 'percent_cols_missing']].rename(columns={'parcelid_x':'id'}, inplace=True)
    

    df_null = pd.DataFrame({'missing_vals': num_missing, 'percent': pct_miss})

    df_nulls = df.merge(df_null,
                        left_index=True,
                        right_index=True).reset_index()[[index_id, 'missing_vals', 'percent']]
    
    return df_nulls.sort_values(by='missing_vals', ascending=False)

In [48]:
df_nulls = nulls_by_row(df)
df_nulls.head()

,parcelid,missing_vals,percent
52441,167687839,48,71.0
52440,167686999,48,71.0
50123,17188862,47,69.0
46567,14460039,46,68.0
44136,14359299,45,66.0


In [ ]:
# mass removal of columns
def remove_columns(df, cols_to_remove):
    """
    This function will:
    - take in a df and list of columns
    - drop the listed columns
    - return the new df
    """
    df = df.drop(columns=cols_to_remove)
    return df

In [ ]:
# handles large df with defaults to remove if over threshold
def handle_missing_values(df, prop_required_columns=0.5, prop_required_rows=0.75):
    """
    This function will:
    - take in: 
        - a dataframe
        - column threshold (defaulted to 0.5)
        - row threshold (defaulted to 0.75)
    - calculates the minimum number of non-missing values required for each column/row to be retained
    - drops columns/rows with a high proportion of missing values.
    - returns the new df
    """
    
    column_threshold = int(round(prop_required_columns * len(df.index), 0))
    df = df.dropna(axis=1, thresh=column_threshold)
    
    row_threshold = int(round(prop_required_rows * len(df.columns), 0))
    df = df.dropna(axis=0, thresh=row_threshold)
    
    return df

In [ ]:
# not really sure yet what this one is for
def data_prep(df, col_to_remove=[], prop_required_columns=0.5, prop_required_rows=0.75):
    """
    This function will:
    - take in: 
        - a dataframe
        - list of columns
        - column threshold (defaulted to 0.5)
        - row threshold (defaulted to 0.75)
    - removes unwanted columns
    - remove rows and columns that contain a high proportion of missing values
    - returns cleaned df
    """
    df = remove_columns(df, col_to_remove)
    df = handle_missing_values(df, prop_required_columns, prop_required_rows)
    return df